In [1]:
%%capture
import tensorflow as tf
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
tf.__version__

## Define tfkeras model

Only tfkeras tf-2.0.0+😉

In [2]:
def BlazeBlock(x, filters=16, kernel_size=3, strides=1, padding='same', name=None):
    
    
    x_l = DepthwiseConv2D(        
        kernel_size=kernel_size,
        strides=strides,
        padding=padding,
        use_bias=True)(x)
    
    # 1x1 
    x_l = Conv2D(
        filters=filters,
        kernel_size=1,
        strides=1,
        padding=padding,
        use_bias=True)(x_l)

    
    # max pooling
    if strides == 2:       
        input_channels = x.shape[-1]
        output_channels = x_l.shape[-1]

        
        x_r = MaxPooling2D(padding='same')(x)
      

        # channel padding
        num_channel_pads = output_channels - input_channels
        if num_channel_pads != 0:
            pads = tf.constant([[0, 0], [0, 0], [0, 0], [0, int(num_channel_pads)]])
            x_r = tf.pad(x_r, pads, "CONSTANT")


        out = Add()([x_l, x_r])
        return PReLU(shared_axes=[1, 2], name=name)(out)
    else: 
        
        out = Add()([x_l, x])        
        return PReLU(shared_axes=[1, 2], name=name)(out)




In [3]:
def FaceLandmarks(input_shape):

    inputs = tf.keras.layers.Input(shape=input_shape, name='input')

    # dummy ¯\_(ツ)_/¯
    cv0 = Conv2D(3,3)
    dw0 = DepthwiseConv2D(3)
    pr0 = PReLU()
    
    x = Conv2D(filters=16, kernel_size=3, strides=2, padding='valid')(inputs)   
    x = PReLU(shared_axes=[1, 2])(x)  
    x = BlazeBlock(x, filters=16)
    x = BlazeBlock(x, filters=16)

    x = BlazeBlock(x, filters=32, strides=2)
    x = BlazeBlock(x, filters=32)
    x = BlazeBlock(x, filters=32)

    x = BlazeBlock(x, filters=64, strides=2)
    x = BlazeBlock(x, filters=64)
    x = BlazeBlock(x, filters=64)

    x = BlazeBlock(x, filters=128, strides=2)
    x = BlazeBlock(x, filters=128)
    x = BlazeBlock(x, filters=128)

    x = BlazeBlock(x, filters=128, strides=2)
    x = BlazeBlock(x, filters=128)
    x = BlazeBlock(x, filters=128)
    
    
    # keypoints path.
    x_keyp = BlazeBlock(x, filters=128, strides=2)
    x_keyp = BlazeBlock(x_keyp, filters=128)
    x_keyp = BlazeBlock(x_keyp, filters=128)
    x_keyp = Conv2D(kernel_size=1, filters=32, padding='same')(x_keyp)
    x_keyp = PReLU(shared_axes=[1, 2])(x_keyp)
    x_keyp = BlazeBlock(x_keyp, filters=32)
    x_keyp = Conv2D(kernel_size=3, filters=1404)(x_keyp)
    
    
    
    
    # hack way to make the layer numering correct. ¯\_(ツ)_/¯
    dw18 = DepthwiseConv2D(3)
    dw19 = DepthwiseConv2D(3)
    dw20 = DepthwiseConv2D(3)
    dw21 = DepthwiseConv2D(3)
    cv21 = Conv2D(3,3)
    cv22 = Conv2D(3,3)
    cv23 = Conv2D(3,3)
    cv24 = Conv2D(3,3)
    cv25 = Conv2D(3,3)
    cv26 = Conv2D(3,3)
    mx5 = MaxPooling2D()
    pr20 = PReLU()
    pr21 = PReLU()
    pr22 = PReLU()
    pr23 = PReLU()
    pr24 = PReLU()
    
    
   
    
    # face flag path.
    x_flag = BlazeBlock(x, filters=128, strides=2)
    x_flag = Conv2D(kernel_size=1, filters=32, padding='same')(x_flag)
    x_flag = PReLU(shared_axes=[1, 2])(x_flag)
    
    x_flag = BlazeBlock(x_flag, filters=32)
    x_flag = Conv2D(kernel_size=3, filters=1)(x_flag)
    

    
    
    model = tf.keras.models.Model(inputs=inputs, outputs=[x_keyp, x_flag])
    return model


In [4]:
INPUT_SHAPE = (192, 192,3)
model = FaceLandmarks(INPUT_SHAPE)

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 192, 192, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 95, 95, 16)   448         input[0][0]                      
__________________________________________________________________________________________________
p_re_lu_1 (PReLU)               (None, 95, 95, 16)   16          conv2d_1[0][0]                   
__________________________________________________________________________________________________
depthwise_conv2d_1 (DepthwiseCo (None, 95, 95, 16)   160         p_re_lu_1[0][0]                  
______________________________________________________________________________________________

In [11]:
model.save('model/face_landmark.h5')

In [12]:
model.save('model/saved_model', save_format='tf')

## TFLite Weights convert.

In [6]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model/face_landmark_new.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()



In [7]:
all_layers_details = interpreter.get_tensor_details() 

In [8]:
def correct_name(tflite_name):
    # change tflite layer name to keras name.
    if tflite_name.startswith('depthwise'):
        tflite_name = tflite_name.replace('Kernel', 'depthwise_kernel') 
    return tflite_name.lower()+":0"
        

In [9]:
weights_dict = {}
for layer in all_layers_details:
    if ('Kernel' in layer['name'] or 'Bias' in layer['name'] or 'Alpha' in layer['name']):

        keras_layer_name = correct_name(layer['name'])
        print(keras_layer_name)
        w = interpreter.get_tensor(layer['index'])
        weights_dict[keras_layer_name] = w

conv2d_1/kernel:0
conv2d_1/bias:0
p_re_lu_1/alpha:0
depthwise_conv2d_1/depthwise_kernel:0
depthwise_conv2d_1/bias:0
conv2d_2/kernel:0
conv2d_2/bias:0
p_re_lu_2/alpha:0
depthwise_conv2d_2/depthwise_kernel:0
depthwise_conv2d_2/bias:0
conv2d_3/kernel:0
conv2d_3/bias:0
p_re_lu_3/alpha:0
depthwise_conv2d_3/depthwise_kernel:0
depthwise_conv2d_3/bias:0
conv2d_4/kernel:0
conv2d_4/bias:0
p_re_lu_4/alpha:0
depthwise_conv2d_4/depthwise_kernel:0
depthwise_conv2d_4/bias:0
conv2d_5/kernel:0
conv2d_5/bias:0
p_re_lu_5/alpha:0
depthwise_conv2d_5/depthwise_kernel:0
depthwise_conv2d_5/bias:0
conv2d_6/kernel:0
conv2d_6/bias:0
p_re_lu_6/alpha:0
depthwise_conv2d_6/depthwise_kernel:0
depthwise_conv2d_6/bias:0
conv2d_7/kernel:0
conv2d_7/bias:0
p_re_lu_7/alpha:0
depthwise_conv2d_7/depthwise_kernel:0
depthwise_conv2d_7/bias:0
conv2d_8/kernel:0
conv2d_8/bias:0
p_re_lu_8/alpha:0
depthwise_conv2d_8/depthwise_kernel:0
depthwise_conv2d_8/bias:0
conv2d_9/kernel:0
conv2d_9/bias:0
p_re_lu_9/alpha:0
depthwise_conv2d_9/d

In [10]:
for each_layer in model.layers:
    # conv2d
#     print("="*10, each_layer.name)
    
    if hasattr(each_layer, 'depthwise_kernel') and each_layer.depthwise_kernel is not None:    
        #print('depth wise convolution')
        K.set_value(each_layer.weights[0], weights_dict[each_layer.name+'/depthwise_kernel:0'].transpose(1,2,3,0));print(each_layer.name+'.weight')
    
    
    
    if hasattr(each_layer, 'kernel') and each_layer.kernel is not None:        
        if len(weights_dict[each_layer.name+'/kernel:0'].shape) == 4:   
            #print('normal 2d convolution')
            K.set_value(each_layer.weights[0], weights_dict[each_layer.name+'/kernel:0'].transpose(1,2,3,0));print(each_layer.name+'.weight')
        
            
    if hasattr(each_layer, 'bias') and each_layer.bias is not None:
        K.set_value(each_layer.weights[1], weights_dict[each_layer.name+'/bias:0']);print(each_layer.name+'.bias')
        
    if hasattr(each_layer, 'alpha') and each_layer.alpha is not None:
        K.set_value(each_layer.weights[0], weights_dict[each_layer.name+'/alpha:0']);print(each_layer.name+'.alpha')
    
   

conv2d_1.weight
conv2d_1.bias
p_re_lu_1.alpha
depthwise_conv2d_1.weight
depthwise_conv2d_1.bias
conv2d_2.weight
conv2d_2.bias
p_re_lu_2.alpha
depthwise_conv2d_2.weight
depthwise_conv2d_2.bias
conv2d_3.weight
conv2d_3.bias
p_re_lu_3.alpha
depthwise_conv2d_3.weight
depthwise_conv2d_3.bias
conv2d_4.weight
conv2d_4.bias
p_re_lu_4.alpha
depthwise_conv2d_4.weight
depthwise_conv2d_4.bias
conv2d_5.weight
conv2d_5.bias
p_re_lu_5.alpha
depthwise_conv2d_5.weight
depthwise_conv2d_5.bias
conv2d_6.weight
conv2d_6.bias
p_re_lu_6.alpha
depthwise_conv2d_6.weight
depthwise_conv2d_6.bias
conv2d_7.weight
conv2d_7.bias
p_re_lu_7.alpha
depthwise_conv2d_7.weight
depthwise_conv2d_7.bias
conv2d_8.weight
conv2d_8.bias
p_re_lu_8.alpha
depthwise_conv2d_8.weight
depthwise_conv2d_8.bias
conv2d_9.weight
conv2d_9.bias
p_re_lu_9.alpha
depthwise_conv2d_9.weight
depthwise_conv2d_9.bias
conv2d_10.weight
conv2d_10.bias
p_re_lu_10.alpha
depthwise_conv2d_10.weight
depthwise_conv2d_10.bias
conv2d_11.weight
conv2d_11.bias
p_r